In [75]:
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt

In [76]:
traindf_x = pd.read_csv("../data/train.csv", index_col="ID")
traindf_y = pd.read_csv("../data/train_label.csv")
testdf_x = pd.read_csv("../data/test.csv", index_col="ID")
testdf_y = pd.read_csv("../data/test_nolabel.csv")

adr = pd.read_csv("../data/TestAdr.csv")
is_canceled = pd.read_csv("../data/TestIsCanceled.csv")

In [84]:
testdf_x["adr"] = adr["adr"]
testdf_x["is_canceled"] = is_canceled["is_canceled"]

In [77]:
traindf_x.sample(2)

,ID,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
90126,90126,City Hotel,0,16,2017,March,12,23,0,1,...,No Deposit,NaN,NaN,0,Transient-Party,99.516653,0,0,Check-Out,2017-03-24
82701,82701,Resort Hotel,0,15,2017,February,5,3,0,2,...,No Deposit,240.0,NaN,0,Transient,68.566599,0,1,Check-Out,2017-02-05


In [78]:
def create_datetime(year: pd.Series, month: pd.Series, date: pd.Series) -> pd.Series:
    return pd.to_datetime(year.astype(str) + month + date.astype(str), format="%Y%B%d")

def year_month_to_date(df):
    df["date"] = create_datetime(df["arrival_date_year"], df["arrival_date_month"], df["arrival_date_day_of_month"])
    df.drop(["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month"], 
        axis=1, inplace=True)
    return df

In [79]:
traindf_x = year_month_to_date(traindf_x)
testdf_x = year_month_to_date(testdf_x)

traindf_y["arrival_date"] = pd.to_datetime(traindf_y["arrival_date"])
testdf_y["arrival_date"] = pd.to_datetime(testdf_y["arrival_date"])

In [80]:
traindf_x.sample(2)

,ID,hotel,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,date
8550,8550,Resort Hotel,0,119,37,2,4,2,0.0,0,...,240.0,NaN,0,Transient,80.985945,0,1,Check-Out,2015-09-18,2015-09-12
75449,75449,City Hotel,0,16,50,2,0,2,1.0,0,...,9.0,NaN,0,Transient,145.635763,0,1,Check-Out,2016-12-06,2016-12-04


In [81]:
def stay_nights(df):
    return df["stays_in_weekend_nights"] + df["stays_in_week_nights"]

traindf_x["stay_nights"] = stay_nights(traindf_x)
testdf_x["stay_nights"] = stay_nights(testdf_x)

traindf_x = traindf_x[traindf_x["stay_nights"] != 0]
testdf_x = testdf_x[testdf_x["stay_nights"] != 0]

In [82]:
def explode_date(df):
    df['date'] = [pd.date_range(s, periods=e, freq='d') for s, e in
              zip(df['date'], df['stay_nights'])]
    df = df.explode("date")
    return df

traindf_x = explode_date(traindf_x)
testdf_x = explode_date(testdf_x)

In [85]:
def true_adr(df):
    return df["adr"] * (1 - df["is_canceled"])

traindf_x["adr"] = true_adr(traindf_x)
testdf_x["adr"] = true_adr(testdf_x)

In [110]:
traindf = traindf_y.merge(traindf_x.groupby("date").agg("sum")["adr"].reset_index(level=0), how="left",
         left_on="arrival_date", right_on="date")
testdf = testdf_y.merge(testdf_x.groupby("date").agg("sum")["adr"].reset_index(level=0), how="left",
         left_on="arrival_date", right_on="date")

In [159]:
testdf.sample(2)

,arrival_date,date,adr
55,2017-05-26,2017-05-26,20593.350426
3,2017-04-04,2017-04-04,17299.821347


In [126]:
traindf = traindf.loc[20:]

In [137]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import seaborn as sns
import sklearn
import statsmodels.api as sm
import xgboost as xgb
from matplotlib import pyplot
from matplotlib.font_manager import FontProperties
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis,
                                           QuadraticDiscriminantAnalysis)
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              ExtraTreesClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, VotingClassifier)
from sklearn.feature_selection import (RFE, SelectFromModel, SelectKBest,
                                       VarianceThreshold, chi2)
from sklearn.inspection import permutation_importance
from sklearn.linear_model import (LinearRegression, LogisticRegression, PassiveAggressiveClassifier,
                                  Perceptron, RidgeClassifier, SGDClassifier)
from sklearn.metrics import (accuracy_score, balanced_accuracy_score, mean_absolute_error,
                             classification_report, confusion_matrix, log_loss,
                             precision_score, recall_score, roc_auc_score)
from sklearn.model_selection import (GridSearchCV, KFold, RandomizedSearchCV,
                                     ShuffleSplit, StratifiedKFold,
                                     cross_val_score, train_test_split)
from sklearn.naive_bayes import (BernoulliNB, ComplementNB, GaussianNB,
                                 MultinomialNB)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (Binarizer, LabelEncoder, MinMaxScaler,
                                   Normalizer, OneHotEncoder,
                                   PolynomialFeatures, StandardScaler,
                                   normalize)
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [156]:
new_train_x = traindf[["adr"]]
new_train_y = traindf["label"].replace([6, 7, 8, 9], 6).astype("int64")

In [157]:
X_train, X_test, y_train, y_test = train_test_split(new_train_x, new_train_y, test_size=0.3, random_state=42)

In [158]:
classifiers = [XGBClassifier(),
               ExtraTreesClassifier(),
               DecisionTreeClassifier(),
               RandomForestClassifier(),
               RidgeClassifier(),
               QuadraticDiscriminantAnalysis(),
               ComplementNB(),
               BernoulliNB(),
               GaussianNB(),
               MultinomialNB(),
               LinearDiscriminantAnalysis(),
               SGDClassifier(),
               AdaBoostClassifier(),
               BaggingClassifier(),
               LogisticRegression(),
               LinearSVC(),
               KNeighborsClassifier()
               ]

from time import time

for clf in classifiers:
    
    before = time()
    print("="*30)
    print(clf.__class__.__name__)
    
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    print(f"Mean absolute error: {mean_absolute_error(y_test, y_pred):.2f}")
    after = time()
    print(f"花費{(after - before):.1f}s")
    
print("="*30)

XGBClassifier
[14:40:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Mean absolute error: 0.86
花費0.3s
ExtraTreesClassifier
Mean absolute error: 0.81
花費0.1s
DecisionTreeClassifier
Mean absolute error: 0.84
花費0.0s
RandomForestClassifier
Mean absolute error: 0.84
花費0.1s
RidgeClassifier
Mean absolute error: 0.72
花費0.0s
QuadraticDiscriminantAnalysis
Mean absolute error: 0.64
花費0.0s
ComplementNB
Mean absolute error: 2.03
花費0.0s
BernoulliNB
Mean absolute error: 1.05
花費0.0s
GaussianNB
Mean absolute error: 0.65
花費0.0s
MultinomialNB
Mean absolute error: 1.05
花費0.0s
LinearDiscriminantAnalysis
Mean absolute error: 0.63
花費0.0s
SGDClassifier
Mean absolute error: 1.46
花費0.0s
AdaBoostClassifier
Mean absolute error: 0.70
花費0.1s
BaggingClassifier
Mean absolute error: 0.82
花費0.0s
LogisticRegression
Mean absolute error: 1.05
花費0.0s
LinearSVC
Mean absolute error: 2.03
花費0.0s
KNeighborsClassifier
Mean absolute error: 0.77
花費0.0s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [162]:
clf = GaussianNB()
clf.fit(new_train_x, new_train_y)
y_pred = clf.predict(testdf[["adr"]])

In [164]:
testdf_y.merge()

153

153